In [1]:
# 1. 필요한 라이브러리 임포트
import os
import psycopg2
import pandas as pd
import numpy as np
from datetime import datetime
import json
from dotenv import load_dotenv
from openai import AzureOpenAI
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

# .env 파일 로드
load_dotenv()

print("라이브러리 임포트 완료")

라이브러리 임포트 완료


In [2]:
# 2. PostgreSQL 연결 설정
def get_db_connection():
    """PostgreSQL 데이터베이스 연결"""
    try:
        conn = psycopg2.connect(
            host=os.getenv('PG_HOST'),
            port=os.getenv('PG_PORT'),
            database=os.getenv('PG_DATABASE'),
            user=os.getenv('PG_USER'),
            password=os.getenv('PG_PASSWORD')
        )
        print(f"✅ PostgreSQL 연결 성공: {os.getenv('PG_HOST')}")
        return conn
    except Exception as e:
        print(f"❌ PostgreSQL 연결 실패: {e}")
        return None

# 연결 테스트
conn = get_db_connection()
if conn:
    conn.close()

✅ PostgreSQL 연결 성공: dev-rubicon-postgresql.postgres.database.azure.com


In [3]:
# 3. Azure OpenAI 클라이언트 설정
def get_openai_client():
    """Azure OpenAI 클라이언트 생성"""
    try:
        # 환경 변수 확인
        endpoint = os.getenv('ENDPOINT_URL')
        api_key = os.getenv('AZURE_OPENAI_API_KEY')
        api_version = os.getenv('AZURE_API_VERSION', '2024-02-01')  # 기본값 제공
        
        if not endpoint:
            print("❌ ENDPOINT_URL 환경 변수가 설정되지 않았습니다.")
            return None
        
        if not api_key:
            print("❌ AZURE_OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
            return None
        
        # API 버전이 없으면 기본값 사용
        if not api_version:
            api_version = '2024-02-01'
            print(f"⚠️ AZURE_API_VERSION이 설정되지 않아 기본값 사용: {api_version}")
        
        print(f"📋 Azure OpenAI 설정:")
        print(f"  - Endpoint: {endpoint[:50]}...")
        print(f"  - API Version: {api_version}")
        print(f"  - Deployment: {os.getenv('DEPLOYMENT_NAME')}")
        
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            api_key=api_key,
            api_version=api_version,
        )
        print(f"✅ Azure OpenAI 클라이언트 생성 성공")
        return client
    except Exception as e:
        print(f"❌ Azure OpenAI 클라이언트 생성 실패: {e}")
        return None

# 클라이언트 테스트
openai_client = get_openai_client()

# 간단한 테스트 호출
if openai_client:
    try:
        print("\n🧪 API 연결 테스트...")
        test_response = openai_client.chat.completions.create(
            model=os.getenv('DEPLOYMENT_NAME'),
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Say 'Hello' in one word."}
            ],
            max_tokens=10
        )
        if test_response and test_response.choices:
            print(f"✅ API 테스트 성공: '{test_response.choices[0].message.content}'")
        else:
            print("⚠️ API 응답이 비어있습니다.")
    except Exception as e:
        print(f"❌ API 테스트 실패: {e}")
        print(f"   상세 에러: {type(e).__name__}")
        if hasattr(e, '__dict__'):
            print(f"   에러 속성: {e.__dict__}")

📋 Azure OpenAI 설정:
  - Endpoint: https://kt-azure-openai-dev-donghunseo.openai.azur...
  - API Version: 2024-02-01
  - Deployment: gpt-4.1
✅ Azure OpenAI 클라이언트 생성 성공

🧪 API 연결 테스트...
✅ API 테스트 성공: 'Hello!'


In [4]:
# 4. 테이블 스키마 정보 조회
def get_table_schema(table_name='test'):
    """테이블 스키마 정보 조회 (코멘트 포함)"""
    conn = get_db_connection()
    if not conn:
        return None
    
    try:
        # PostgreSQL에서 컬럼 정보와 코멘트를 함께 조회
        query = """
        SELECT 
            c.column_name,
            c.data_type,
            c.character_maximum_length,
            c.numeric_precision,
            c.numeric_scale,
            c.is_nullable,
            c.column_default,
            pgd.description as column_comment
        FROM information_schema.columns c
        LEFT JOIN pg_catalog.pg_statio_all_tables as st
            ON c.table_schema = st.schemaname 
            AND c.table_name = st.relname
        LEFT JOIN pg_catalog.pg_description pgd 
            ON pgd.objoid = st.relid 
            AND pgd.objsubid = c.ordinal_position
        WHERE c.table_schema = 'public' 
        AND c.table_name = %s
        ORDER BY c.ordinal_position;
        """
        
        df_schema = pd.read_sql_query(query, conn, params=(table_name,))
        print(f"✅ 테이블 '{table_name}' 스키마 조회 성공")
        print(f"   - 컬럼 수: {len(df_schema)}")
        
        # 코멘트가 있는 컬럼 수 확인
        comment_count = df_schema['column_comment'].notna().sum()
        print(f"   - 코멘트가 있는 컬럼: {comment_count}개")
        
        return df_schema
    
    except Exception as e:
        print(f"❌ 테이블 스키마 조회 실패: {e}")
        return None
    
    finally:
        conn.close()

# 스키마 조회
df_schema = get_table_schema(table_name="kt_merged_product_20250929")
if df_schema is not None:
    print("\n테이블 스키마 정보:")
    display(df_schema.head(10))

✅ PostgreSQL 연결 성공: dev-rubicon-postgresql.postgres.database.azure.com
✅ 테이블 'kt_merged_product_20250929' 스키마 조회 성공
   - 컬럼 수: 34
   - 코멘트가 있는 컬럼: 34개

테이블 스키마 정보:


,column_name,data_type,character_maximum_length,numeric_precision,numeric_scale,is_nullable,column_default,column_comment
0,display_category_major,character varying,1000.0,NaN,NaN,YES,None,"웹사이트에 노출되는 제품의 대분류 카테고리명입니다. (예: 모바일, TV & 오디오..."
1,display_category_middle,character varying,1000.0,NaN,NaN,YES,None,"웹사이트에 노출되는 제품의 중분류 카테고리명입니다. (예: 스마트폰, QLED, 비..."
2,display_category_minor,character varying,1000.0,NaN,NaN,YES,None,"웹사이트에 노출되는 제품의 소분류 카테고리명입니다. (예: Galaxy S, Neo..."
3,product_category_major,character varying,1000.0,NaN,NaN,YES,None,내부 시스템에서 관리하는 제품의 대분류 카테고리 코드 또는 이름입니다.
4,product_category_middle,character varying,1000.0,NaN,NaN,YES,None,내부 시스템에서 관리하는 제품의 중분류 카테고리 코드 또는 이름입니다.
5,product_category_minor,character varying,1000.0,NaN,NaN,YES,None,내부 시스템에서 관리하는 제품의 소분류 카테고리 코드 또는 이름입니다.
6,model_name,character varying,1000.0,NaN,NaN,YES,None,제품의 공식 모델명입니다. 사용자가 일반적으로 인지하는 이름입니다. (예: Gala...
7,model_code,character varying,1000.0,NaN,NaN,YES,None,"제품의 세부 사양(색상, 용량 등)을 포함하는 고유 모델 코드입니다. (예: SM-..."
8,product_id,character varying,1000.0,NaN,NaN,YES,None,데이터베이스에서 각 제품을 식별하기 위한 고유 ID입니다.
9,product_name,character varying,1000.0,NaN,NaN,YES,None,웹사이트에 최종적으로 표시되는 제품명입니다. (예: 갤럭시 S24 Ultra 자급제...


In [7]:
# 5. kt_merged_product_20250929 테이블 데이터 조회
def get_product_data():
    """kt_merged_product_20250929 테이블에서 특정 컬럼만 조회"""
    conn = get_db_connection()
    if not conn:
        return None
    
    try:
        query = """
        SELECT 
            product_id,
            display_category_major,
            display_category_middle,
            display_category_minor,
            model_code,
            model_name,
            product_name,
            product_specification
        FROM kt_merged_product_20250929
        """
        
        df = pd.read_sql_query(query, conn)
        print(f"✅ 데이터 조회 성공: {len(df)}개 행")
        return df
    
    except Exception as e:
        print(f"❌ 데이터 조회 실패: {e}")
        return None
    
    finally:
        conn.close()

# 데이터 조회 및 표시
df_products = get_product_data()
if df_products is not None:
    print("\nkt_merged_product_20250929 테이블 데이터 (product_id, model_code, model_name, product_specification):")
    display(df_products)

✅ PostgreSQL 연결 성공: dev-rubicon-postgresql.postgres.database.azure.com
✅ 데이터 조회 성공: 2774개 행

kt_merged_product_20250929 테이블 데이터 (product_id, model_code, model_name, product_specification):


,product_id,display_category_major,display_category_middle,display_category_minor,model_code,model_name,product_name,product_specification
0,G000190421,오디오,JBLㅣ하만카돈ㅣAKGㅣ마크레빈슨,JBL-헤드폰,JBLQUANTUM300BLK,None,JBL QUANTUM 300 게이밍 헤드셋,"{'감도': ['100dB SPL @ 1kHz/1mW'], '기타': ['케이블길이..."
1,G000190423,오디오,JBLㅣ하만카돈ㅣAKGㅣ마크레빈슨,JBL-헤드폰,JBLQUANTUMONEBLK,JBLQONE,JBL QUANTUM ONE 게이밍 헤드셋,"{'색상': ['블랙'], 'Weight (g)': ['369 g'], 'Drive..."
2,G000191339,리빙가전,리빙가전 소모품/액세서리,공기청정기 액세서리,CFX-C100D,CFX-C100D,블루스카이 7000 일체형 필터,"{'색상': ['블랙'], '용도': ['공기청정기용'], '제형': ['필터형']..."
3,G000191365,리빙가전,리빙가전 소모품/액세서리,에어컨 액세서리,AF-A1J1D,AF-A1J1D,Bespoke 무풍에어컨 갤러리 아트패널 (갤러리 Ⅰ),"{'색상': ['그리너리', '그리너리'], '제품 중량': ['0.52 kg'],..."
4,G000191380,리빙가전,리빙가전 소모품/액세서리,에어컨 액세서리,AF-C3Y1D,AF-C3Y1D,Bespoke 무풍에어컨 클래식 바람문 아트패널,"{'색상': ['썬 옐로우', '썬 옐로우'], '제품 중량': ['0.2 kg']..."
...,...,...,...,...,...,...,...,...
2769,G100177961,모바일,갤럭시 액세서리,기타,EJ-PN980BJEGKR,EJ-PN980,갤럭시 노트20｜20 울트라 S펜,"{'색상': ['그레이', '그레이'], '중량': ['3 g'], '특징': ['..."
2770,G100178765,모바일,갤럭시 액세서리,충전기,EP-P6300TBEGKR,EP-P6300,무선 충전 트리오,"{'색상': ['블랙', '블랙'], '중량': ['320 g'], '특징': ['..."
2771,G100178766,모바일,갤럭시 액세서리,충전기,EP-P6300TWEGKR,EP-P6300,무선 충전 트리오,"{'색상': ['화이트', '화이트'], '중량': ['320 g'], '특징': ..."
2772,G100179607,주방가전,주방가전 소모품/액세서리,BESPOKE 냉장고 패널,RA-F18DBB35,RA-F18DBB35,Bespoke 냉장고 4도어 패널 (하칸),"{'색상': ['글램 화이트', '글램 화이트'], '주요특징': ['냉동실 좌/우..."


In [11]:
# 6. product_specification JSON 분석
def analyze_product_specifications():
    """product_specification JSON 필드를 상세 분석"""
    
    conn = get_db_connection()
    if not conn:
        return None
    
    try:
        # display_category_middle과 product_specification을 함께 조회
        query = """
        SELECT 
            display_category_middle,
            product_specification
        FROM kt_merged_product_20250929
        WHERE product_specification IS NOT NULL
        """
        
        df = pd.read_sql_query(query, conn)
        print(f"✅ 분석용 데이터 조회 성공: {len(df)}개 행\n")
        
        # 1. display_category_middle별 key-value 분석
        category_key_stats = {}
        all_keys = set()
        key_value_examples = {}
        
        for idx, row in df.iterrows():
            category = row['display_category_middle']
            spec = row['product_specification']
            
            if category not in category_key_stats:
                category_key_stats[category] = {
                    'keys': set(),
                    'key_counts': {},
                    'value_examples': {}
                }
            
            try:
                if isinstance(spec, str):
                    spec_dict = json.loads(spec)
                elif isinstance(spec, dict):
                    spec_dict = spec
                else:
                    continue
                    
                for key, value in spec_dict.items():
                    all_keys.add(key)
                    category_key_stats[category]['keys'].add(key)
                    
                    # key별 카운트
                    if key not in category_key_stats[category]['key_counts']:
                        category_key_stats[category]['key_counts'][key] = 0
                    category_key_stats[category]['key_counts'][key] += 1
                    
                    # value 예시 수집 (최대 3개)
                    if key not in category_key_stats[category]['value_examples']:
                        category_key_stats[category]['value_examples'][key] = set()
                    if len(category_key_stats[category]['value_examples'][key]) < 20:
                        if value and str(value).strip():
                            category_key_stats[category]['value_examples'][key].add(str(value)[:50])
                    
                    # 전체 key-value 예시
                    if key not in key_value_examples:
                        key_value_examples[key] = set()
                    # if len(key_value_examples[key]) < 5:
                    #     if value and str(value).strip():
                    #         key_value_examples[key].add(str(value)[:50])
                    if value and str(value).strip():
                        key_value_examples[key].add(str(value)[:50])
                            
            except Exception as e:
                continue
        
        # 2. display_category_middle별 주요 key 출력
        print("=" * 80)
        print("1. display_category_middle별 주요 Key와 Value 예시")
        print("=" * 80)
        
        for category in sorted(category_key_stats.keys())[:]:  
            stats = category_key_stats[category]
            total_products = sum(stats['key_counts'].values()) / len(stats['keys']) if stats['keys'] else 0
            
            print(f"\n📁 {category}")
            print(f"   총 제품 수: ~{int(total_products)}")
            print(f"   고유 key 수: {len(stats['keys'])}")
            
            top_keys = sorted(stats['key_counts'].items(), key=lambda x: x[1], reverse=True)[:]
            for key, count in top_keys:
                examples = list(stats['value_examples'].get(key, []))[:2]
                examples_str = ", ".join([f"'{ex}'" for ex in examples])
                print(f"   • {key}: {count}회 (예: {examples_str})")
        
        # 3. 특정 key가 어떤 display_category_middle에 속하는지 분석
        print("\n" + "=" * 80)
        print("2. 주요 Key별 소속 Category 분석")
        print("=" * 80)
        
        key_to_categories = {}
        for category, stats in category_key_stats.items():
            for key in stats['keys']:
                if key not in key_to_categories:
                    key_to_categories[key] = set()
                key_to_categories[key].add(category)
        
        # 여러 카테고리에 걸쳐 있는 key들
        cross_category_keys = {k: v for k, v in key_to_categories.items() if len(v) > 1}
        sorted_keys = sorted(cross_category_keys.items(), key=lambda x: len(x[1]), reverse=True)[:]
        
        print("\n📊 여러 카테고리에 걸쳐 있는 Key (상위 10개):")
        for key, categories in sorted_keys:
            print(f"   • {key}: {len(categories)}개 카테고리")
            print(f"     → {', '.join(list(categories)[:])}")
        
        # 단일 카테고리에만 있는 특화된 key들
        single_category_keys = {k: v for k, v in key_to_categories.items() if len(v) == 1}
        
        print("\n📌 특정 카테고리 전용 Key 예시:")
        category_specific = {}
        for key, categories in single_category_keys.items():
            cat = list(categories)[0]
            if cat not in category_specific:
                category_specific[cat] = []
            category_specific[cat].append(key)
        
        for cat, keys in list(category_specific.items())[:5]:
            print(f"   • {cat}: {', '.join(keys[:3])}")
        
        # 4. 전체 통계
        print("\n" + "=" * 80)
        print("3. 전체 JSON Key-Value 통계")
        print("=" * 80)
        
        # key별 전체 사용 횟수
        total_key_counts = {}
        for stats in category_key_stats.values():
            for key, count in stats['key_counts'].items():
                if key not in total_key_counts:
                    total_key_counts[key] = 0
                total_key_counts[key] += count
        
        print(f"\n📈 전체 통계:")
        print(f"   • 총 고유 key 수: {len(all_keys)}")
        print(f"   • 총 카테고리 수: {len(category_key_stats)}")
        print(f"   • 분석된 제품 수: {len(df)}")
        
        print(f"\n📊 가장 많이 사용되는 Key :")
        top_overall_keys = sorted(total_key_counts.items(), key=lambda x: x[1], reverse=True)[:]
        for key, count in top_overall_keys:
            percentage = (count / len(df)) * 100
            examples = list(key_value_examples.get(key, []))[:2]
            examples_str = ", ".join([f"'{ex}'" for ex in examples])
            print(f"   {key:30s} : {count:5d}회 ({percentage:5.1f}%) | 예: {examples_str}")
        
        # 5. 데이터 타입 분석
        print(f"\n📝 Value 데이터 타입 패턴:")
        type_patterns = {
            '숫자형': [],
            '불린형': [],
            '텍스트형': [],
            '단위포함': [],
            '리스트형': []
        }
        
        for key, examples in key_value_examples.items():
            sample = list(examples)[0] if examples else ""
            if sample:
                if sample.lower() in ['true', 'false', '예', '아니오']:
                    type_patterns['불린형'].append(key)
                elif any(unit in sample for unit in ['GB', 'MB', 'mm', 'cm', 'kg', 'W', 'Hz', 'mAh']):
                    type_patterns['단위포함'].append(key)
                elif sample.replace('.', '').replace('-', '').isdigit():
                    type_patterns['숫자형'].append(key)
                elif ',' in sample or '|' in sample:
                    type_patterns['리스트형'].append(key)
                else:
                    type_patterns['텍스트형'].append(key)
        
        for pattern_type, keys in type_patterns.items():
            if keys:
                print(f"   • {pattern_type}: {', '.join(keys[:5])}")
        
        return df
        
    except Exception as e:
        print(f"❌ 분석 실패: {e}")
        return None
    
    finally:
        conn.close()

# 분석 실행
df_spec_analysis = analyze_product_specifications()

✅ PostgreSQL 연결 성공: dev-rubicon-postgresql.postgres.database.azure.com
✅ 분석용 데이터 조회 성공: 2412개 행

1. display_category_middle별 주요 Key와 Value 예시

📁 JBLㅣ하만카돈ㅣAKGㅣ마크레빈슨
   총 제품 수: ~8
   고유 key 수: 452
   • 제품명: 82회 (예: '['JBL Authentics 500']', '['JBL TUNE 770NC']')
   • 제조자/수입자: 82회 (예: '['Harman International Industries, Inc./ 삼성전자㈜']', '['Harman International Industries, Incorporated / ')
   • 동일모델의 출시년월: 82회 (예: '['2024년 2월']', '['2022년 4월(국내 기준)']')
   • A/S 책임자와 전화번호: 82회 (예: '['하만오디오 전문 서비스 센터 02-553-3494, 삼성전자 서비스 1588-3366 ', '['하만오디오 전문 서비스 센터02-553-3494']')
   • 드라이버 크기: 72회 (예: '['32 mm']', '['43 x 47mm']')
   • 주파수 응답: 71회 (예: '['50Hz ~ 20kHz(-6dB)']', '['• 패시브 : 20 Hz~40 kHz• 액티브 : 20 Hz~20 kHz']')
   • 제조국가: 61회 (예: '['중국,베트남']', '['중국, 베트남']')
   • 품질보증기준: 59회 (예: '['결함·하자 등에 따른 소비자 피해에 대해서는 소비자분쟁해결기준 (소비자기본법 제16조)')
   • 품질보증서: 54회 (예: '['포함']')
   • 블루투스 프로필 버전: 53회 (예: '['A2DP 1.3, AVRCP 1.6']', '['HFP v1.7.2, AVRCP v1.6.2, A2DP v1.3.2']')
   • 감도: 52회 (예: '['100dB SPL @ 1

In [13]:
# 7. product_specification 고유 key 추출 및 유사 key 분석
def analyze_similar_keys():
    """product_specification의 모든 고유 key를 추출하고 유사한 key 쌍을 찾기"""
    
    conn = get_db_connection()
    if not conn:
        return None
    
    try:
        # 모든 product_specification 데이터 가져오기
        query = """
        SELECT product_specification
        FROM kt_merged_product_20250929
        WHERE product_specification IS NOT NULL
        """
        
        df = pd.read_sql_query(query, conn)
        print(f"✅ 데이터 조회 성공: {len(df)}개 행\n")
        
        # 모든 고유 key 추출
        all_keys = set()
        for spec in df['product_specification']:
            try:
                if isinstance(spec, str):
                    spec_dict = json.loads(spec)
                elif isinstance(spec, dict):
                    spec_dict = spec
                else:
                    continue
                    
                all_keys.update(spec_dict.keys())
            except:
                continue
        
        # 정렬된 key 리스트
        sorted_keys = sorted(list(all_keys))
        
        print("=" * 80)
        print(f"1. 전체 고유 Key 목록 (총 {len(sorted_keys)}개)")
        print("=" * 80)
        
        # 알파벳/한글별로 그룹화
        english_keys = [k for k in sorted_keys if k and k[0].isascii()]
        korean_keys = [k for k in sorted_keys if k and not k[0].isascii()]
        
        print("\n📌 영문 Key (한 줄씩):")
        print("-" * 40)
        for key in english_keys:
            print(key)
        
        print("\n📌 한글 Key (한 줄씩):")
        print("-" * 40)
        for key in korean_keys:
            print(key)
        
        # OpenAI를 사용한 유사 key 분석 (여러 번 호출)
        print("\n" + "=" * 80)
        print("2. OpenAI를 활용한 유사 Key 상세 분석")
        print("=" * 80)
        
        # OpenAI 클라이언트 가져오기
        openai_client = get_openai_client()
        
        all_similar_pairs = []
        
        if openai_client:
            try:
                # key를 배치로 나누어 분석 (한 번에 너무 많으면 API가 제대로 분석 못함)
                batch_size = 50
                for i in range(0, len(sorted_keys), batch_size):
                    batch_keys = sorted_keys[i:i+batch_size]
                    keys_str = ', '.join(batch_keys)
                    
                    print(f"\n🤖 배치 {i//batch_size + 1}/{(len(sorted_keys)-1)//batch_size + 1} 분석 중...")
                    
                    prompt = f"""다음은 제품 사양을 나타내는 JSON key 목록입니다:
{keys_str}

위 key들 중에서 의미가 유사하거나 중복되는 key 쌍들을 모두 찾아주세요.
예를 들어:
- 같은 속성을 다른 이름으로 표현한 경우
- 한글과 영문으로 중복된 경우
- 약어와 전체 이름이 함께 있는 경우
- 대소문자만 다른 경우
- 띄어쓰기나 언더스코어 차이만 있는 경우

가능한 모든 유사 쌍을 찾아서 JSON 형식으로 응답해주세요:
{{
  "similar_pairs": [
    {{"key1": "첫번째key", "key2": "두번째key", "reason": "유사한 이유"}},
    ...
  ]
}}
"""
                    
                    response = openai_client.chat.completions.create(
                        model=os.getenv('DEPLOYMENT_NAME'),
                        messages=[
                            {"role": "system", "content": "You are a data analyst expert in finding similar or duplicate fields in datasets. Find ALL possible similar pairs. Respond in Korean."},
                            {"role": "user", "content": prompt}
                        ],
                        max_tokens=3000,
                        temperature=0.2
                    )
                    
                    if response and response.choices:
                        result_text = response.choices[0].message.content
                        
                        # JSON 파싱 시도
                        try:
                            if '```json' in result_text:
                                json_str = result_text.split('```json')[1].split('```')[0]
                            elif '{' in result_text:
                                start = result_text.index('{')
                                end = result_text.rindex('}') + 1
                                json_str = result_text[start:end]
                            else:
                                json_str = result_text
                                
                            similar_data = json.loads(json_str)
                            
                            if 'similar_pairs' in similar_data:
                                all_similar_pairs.extend(similar_data['similar_pairs'])
                                
                        except json.JSONDecodeError:
                            pass
                
                # 중복 제거 및 출력
                print("\n📊 발견된 모든 유사 Key 쌍 (상세):")
                print("-" * 60)
                
                seen_pairs = set()
                for idx, pair in enumerate(all_similar_pairs, 1):
                    key1, key2 = pair.get('key1', ''), pair.get('key2', '')
                    pair_tuple = tuple(sorted([key1, key2]))
                    
                    if pair_tuple not in seen_pairs:
                        seen_pairs.add(pair_tuple)
                        print(f"\n{idx}. 유사 쌍:")
                        print(f"   Key 1: {key1}")
                        print(f"   Key 2: {key2}")
                        print(f"   이유: {pair.get('reason', '')}")
                        
            except Exception as e:
                print(f"❌ OpenAI API 호출 실패: {e}")
        else:
            print("⚠️ OpenAI 클라이언트를 사용할 수 없습니다.")
        
        # 상세한 규칙 기반 유사성 분석
        print("\n" + "=" * 80)
        print("3. 규칙 기반 유사 Key 상세 탐지")
        print("=" * 80)
        
        similar_groups = {
            '색상 관련': [],
            '크기/용량 관련': [],
            '무게 관련': [],
            '화면 관련': [],
            '배터리 관련': [],
            '메모리 관련': [],
            '카메라 관련': [],
            '네트워크/통신 관련': [],
            '프로세서/성능 관련': [],
            '오디오/사운드 관련': [],
            '연결/포트 관련': [],
            '전원 관련': [],
            '센서 관련': [],
            '보안 관련': [],
            '기타 기능': []
        }
        
        for key in sorted_keys:
            key_lower = key.lower()
            
            # 색상 관련
            if any(word in key_lower for word in ['color', '색상', '컬러', 'colour']):
                similar_groups['색상 관련'].append(key)
            
            # 크기/용량 관련
            if any(word in key_lower for word in ['size', '크기', '사이즈', '용량', 'capacity', 'dimension', '치수', 'volume']):
                similar_groups['크기/용량 관련'].append(key)
            
            # 무게 관련
            if any(word in key_lower for word in ['weight', '무게', '중량', 'mass']):
                similar_groups['무게 관련'].append(key)
            
            # 화면 관련
            if any(word in key_lower for word in ['display', '화면', '디스플레이', 'screen', 'panel', '패널', 'lcd', 'oled', 'resolution', '해상도']):
                similar_groups['화면 관련'].append(key)
            
            # 배터리 관련
            if any(word in key_lower for word in ['battery', '배터리', '전지', 'charge', '충전', 'power bank']):
                similar_groups['배터리 관련'].append(key)
            
            # 메모리 관련
            if any(word in key_lower for word in ['memory', '메모리', 'ram', 'storage', '저장', 'rom', 'ssd', 'hdd']):
                similar_groups['메모리 관련'].append(key)
            
            # 카메라 관련
            if any(word in key_lower for word in ['camera', '카메라', '촬영', '렌즈', 'lens', 'photo', '사진', 'video', '동영상']):
                similar_groups['카메라 관련'].append(key)
            
            # 네트워크/통신 관련
            if any(word in key_lower for word in ['network', '네트워크', 'wifi', 'wi-fi', 'bluetooth', '블루투스', 'lte', '5g', '4g', 'cellular']):
                similar_groups['네트워크/통신 관련'].append(key)
            
            # 프로세서/성능 관련
            if any(word in key_lower for word in ['processor', '프로세서', 'cpu', 'gpu', 'chip', '칩', 'performance', '성능']):
                similar_groups['프로세서/성능 관련'].append(key)
            
            # 오디오/사운드 관련
            if any(word in key_lower for word in ['audio', '오디오', 'sound', '사운드', 'speaker', '스피커', 'microphone', '마이크']):
                similar_groups['오디오/사운드 관련'].append(key)
            
            # 연결/포트 관련
            if any(word in key_lower for word in ['port', '포트', 'usb', 'hdmi', 'connector', '연결', 'jack', '잭']):
                similar_groups['연결/포트 관련'].append(key)
            
            # 전원 관련
            if any(word in key_lower for word in ['power', '전원', 'voltage', '전압', 'adapter', '어댑터', 'charger']):
                similar_groups['전원 관련'].append(key)
            
            # 센서 관련
            if any(word in key_lower for word in ['sensor', '센서', 'gyro', '자이로', 'accelerometer', '가속도계']):
                similar_groups['센서 관련'].append(key)
            
            # 보안 관련
            if any(word in key_lower for word in ['security', '보안', 'fingerprint', '지문', 'face', '얼굴', 'lock', '잠금']):
                similar_groups['보안 관련'].append(key)
        
        print("\n📌 주제별 유사 Key 그룹 (전체 목록):")
        print("-" * 60)
        
        for group_name, keys in similar_groups.items():
            if keys:
                print(f"\n【{group_name}】 총 {len(keys)}개")
                print("-" * 40)
                for key in sorted(keys):
                    print(key)
        
        # 대소문자/언더스코어 차이만 있는 key 찾기
        print("\n" + "=" * 80)
        print("4. 형식만 다른 중복 Key 탐지")
        print("=" * 80)
        
        normalized_keys = {}
        for key in sorted_keys:
            # 정규화: 소문자로 변환, 언더스코어/하이픈/공백 제거
            normalized = key.lower().replace('_', '').replace('-', '').replace(' ', '')
            if normalized not in normalized_keys:
                normalized_keys[normalized] = []
            normalized_keys[normalized].append(key)
        
        print("\n📌 형식만 다른 중복 가능 Key:")
        print("-" * 40)
        
        duplicate_count = 0
        for normalized, original_keys in normalized_keys.items():
            if len(original_keys) > 1:
                duplicate_count += 1
                print(f"\n중복 그룹 {duplicate_count}:")
                for key in original_keys:
                    print(f"  - {key}")
        
        if duplicate_count == 0:
            print("형식만 다른 중복 key가 발견되지 않았습니다.")
        
        return sorted_keys
        
    except Exception as e:
        print(f"❌ 분석 실패: {e}")
        return None
    
    finally:
        conn.close()

# 분석 실행
unique_keys = analyze_similar_keys()

✅ PostgreSQL 연결 성공: dev-rubicon-postgresql.postgres.database.azure.com
✅ 데이터 조회 성공: 2412개 행

1. 전체 고유 Key 목록 (총 2208개)

📌 영문 Key (한 줄씩):
----------------------------------------
*제품중량(kg)
100°C 열풍건조 (3단계)
10ml 단위 미세출수
180mm
1회 사용 전력량 표시
2.4G Wi-Fi 네트워크 호환성
2.4G Wi-Fi 변조
2.4G Wi-Fi 송신기 전력
2.4G Wi-Fi 송신기 주파수 범위
2.4G Wi-Fi 송신기 출력
2.4G 무선 변조
2.4G 무선 송수신 출력
2.4G 무선 통신 주파수
2.4GHz 동글을 켜고 ANC를 끈 상태에서 통화 시간
2.4GHz 동글을 켜고 ANC를 킨 상태에서 통화 시간
2.4GHz 무선 변조
2.4GHz 무선 송신기 전력
2.4GHz 무선 통신사 주파수
2.4GHz 켜짐 및 ANC 켜짐 상태에서 통화 시간
2.4GHz를 켜고 ANC를 끈 상태에서 통화 시간
20년형 스튜디오 스탠드 지원
24시간 켜짐/꺼짐 예약
2G CDMA
2G GSM
3.5 mm 오디오 입력
3.5 mm 오디오 케이블 입력
3.5mm Audio Cable Input
3.5mm 오디오 케이블
360 사운드
360 오디오
3D 돌비 애트모스
3D 맵핑
3G CDMA
3G TD-SCDMA
3G UMTS
3단 레일 서랍
4G FDD LTE
4G TDD LTE
4K 영상 출력
4단 입체 물살
5G FDD Sub6
5G SDL Sub6
5G SUL Sub6
5G TDD Sub6
5G TDD mmWave
5G Wi-Fi 네트워크 호환성
5G Wi-Fi 변조
5G Wi-Fi 송신기 전력
5G Wi-Fi 송신기 주파수 범위
5G Wi-Fi 송신기 출력
7.1ch Quantum 서라운드
720° 무빙세척날개 (하단)
A/S 책임자와 전화번호
A/S책임자와 전화번호
AAC
AI 맞춤
AI 맞춤 세척+
AI 모드


In [ ]:
df_products